# Richter's Predictor: Modeling Earthquake Damage

Based on aspects of building location and construction, your goal is to predict the level of damage to buildings caused by the 2015 Gorkha earthquake in Nepal.

### Libraries

In [1]:
# Data exploration and manipulation essentials
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Statistic tests
from scipy import stats
# from statsmodels.stats.outliers_influence import variance_inflation_factor # Calculating VIF

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

# Classification Models
import statsmodels.formula.api as smf
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
# from sklearn.ensemble import VotingClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

# Cross-validation
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer  

# Metrics
from sklearn.metrics import roc_auc_score, confusion_matrix

### Importing Data

In [2]:
# training data
train_values = pd.read_csv('data/train_values.csv')
train_labels = pd.read_csv('data/train_labels.csv')

# testing data
test_values = pd.read_csv('data/test_values.csv')

In [3]:
train_values.drop('building_id', inplace=True, axis=1)
test_values.drop('building_id', inplace=True, axis=1)

### Creating Numeric and Categorical Lists

In [4]:
columns = train_values.columns

for i in columns:
    if len(train_values[i].unique()) == 2:
        train_values[i] = train_values[i].astype('category')

In [5]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   geo_level_1_id                          260601 non-null  int64   
 1   geo_level_2_id                          260601 non-null  int64   
 2   geo_level_3_id                          260601 non-null  int64   
 3   count_floors_pre_eq                     260601 non-null  int64   
 4   age                                     260601 non-null  int64   
 5   area_percentage                         260601 non-null  int64   
 6   height_percentage                       260601 non-null  int64   
 7   land_surface_condition                  260601 non-null  object  
 8   foundation_type                         260601 non-null  object  
 9   roof_type                               260601 non-null  object  
 10  ground_floor_type               

In [6]:
# numeric data frame
numeric_train = train_values.select_dtypes(include=[int, float])

numeric_cols = list(numeric_train.columns)

numeric_test = test_values[numeric_cols]

# categorical data frame
categorical_train = train_values.select_dtypes(exclude=[int, float])

categorical_cols = list(categorical_train.columns)

categorical_test = test_values[categorical_cols]

In [7]:
print(numeric_cols, '\n-----------------------\n', categorical_cols)

['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families'] 
-----------------------
 ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'legal_ownership_status', 'has_secondary_use', 'has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school', 'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office', 'has_secondary_us

In [8]:
numeric_train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,count_families
0,6,487,12198,2,30,6,5,1
1,8,900,2812,2,10,8,7,1
2,21,363,8973,2,10,5,5,1
3,22,418,10694,2,10,6,5,1
4,11,131,1488,3,30,8,9,1


In [9]:
numeric_test.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,count_families
0,17,596,11307,3,20,7,6,1
1,6,141,11987,2,25,13,5,1
2,22,19,10044,2,5,4,5,1
3,26,39,633,1,0,19,3,2
4,17,289,7970,3,15,8,7,1


### Train-Test split the Training Set

In [10]:
X = train_values
y = train_labels.drop('building_id', axis=1).values.ravel() # to convert to a 1-d numpy array

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)

### Feature Scaling Numeric Columns

In [12]:
# scaling the numeric columns
Standard_Scaler = StandardScaler()

In [13]:
# sub training and testing data
xtrain_scaled = Standard_Scaler.fit_transform(xtrain[numeric_cols])
xtest_scaled = Standard_Scaler.transform(xtest[numeric_cols])

xtrain[numeric_cols] = xtrain_scaled
xtest[numeric_cols] = xtest_scaled

In [14]:
# full training and testing data
train_scaled = Standard_Scaler.fit_transform(X[numeric_cols])
X[numeric_cols] = train_scaled

test_scaled = Standard_Scaler.fit_transform(test_values[numeric_cols])
test_values[numeric_cols] = test_scaled

### Feature Scaling Categorical Columns

In [15]:
# one-hot encoding the columns that are not yet numerical
not_yet_numeric = categorical_train.select_dtypes(exclude=['category'])
not_yet_numeric_cols = not_yet_numeric.columns

In [16]:
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'), 
     ['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'legal_ownership_status']))

Sub-Training Set

In [17]:
transformed_xtrain = transformer.fit_transform(xtrain)
dense_array = transformed_xtrain.toarray()
transformed_df = pd.DataFrame(dense_array, columns = transformer.get_feature_names_out())
transformed_df.index = xtrain.index

# Joining tables
xtrain = pd.concat([xtrain, transformed_df], axis=1)
# Dropping old categorical columns
xtrain.drop(not_yet_numeric_cols, axis=1, inplace=True)

In [18]:
xtrain = xtrain.reset_index(drop=True)
xtrain.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,onehotencoder__plan_configuration_m,onehotencoder__plan_configuration_n,onehotencoder__plan_configuration_o,onehotencoder__plan_configuration_q,onehotencoder__plan_configuration_s,onehotencoder__plan_configuration_u,onehotencoder__legal_ownership_status_a,onehotencoder__legal_ownership_status_r,onehotencoder__legal_ownership_status_v,onehotencoder__legal_ownership_status_w
0,-0.734245,0.306735,0.770372,1.198079,-0.225337,-0.003630,1.862323,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-1.232181,-1.349505,1.693148,-0.178324,-0.293740,-0.003630,-0.747645,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-1.232181,-0.415900,1.178634,-0.178324,0.116680,-0.232414,-0.747645,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.386112,-0.510473,0.448870,-0.178324,0.527099,-0.461199,-0.747645,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.381986,1.618631,-0.113692,-0.178324,-0.293740,-0.461199,-0.225652,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Sub-Testing Set

In [19]:
transformed_xtest = transformer.transform(xtest)
dense_array = transformed_xtest.toarray()
transformed_df = pd.DataFrame(dense_array, columns = transformer.get_feature_names_out())
transformed_df.index = xtest.index

# Joining tables
xtest = pd.concat([xtest, transformed_df], axis=1)
# Dropping old categorical columns
xtest.drop(not_yet_numeric_cols, axis=1, inplace=True)

In [20]:
xtest = xtest.reset_index(drop=True)
xtest.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,onehotencoder__plan_configuration_m,onehotencoder__plan_configuration_n,onehotencoder__plan_configuration_o,onehotencoder__plan_configuration_q,onehotencoder__plan_configuration_s,onehotencoder__plan_configuration_u,onehotencoder__legal_ownership_status_a,onehotencoder__legal_ownership_status_r,onehotencoder__legal_ownership_status_v,onehotencoder__legal_ownership_status_w
0,0.012660,1.616206,-0.296270,1.198079,0.253486,-0.689984,0.296342,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.983213,0.105464,-0.140872,-0.178324,-0.225337,-0.232414,-0.225652,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.360792,0.156388,-1.532861,2.574482,0.321889,-0.003630,1.862323,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1.605634,-1.019712,0.007112,-0.178324,0.185083,0.225155,-0.225652,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.858729,-0.352851,-0.371223,-1.554727,-0.293740,-1.147554,-1.791633,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Full Training Data

In [21]:
transformed_X = transformer.fit_transform(X)
dense_array = transformed_X.toarray()
transformed_df = pd.DataFrame(dense_array, columns = transformer.get_feature_names_out())
transformed_df.index = X.index

# Joining tables
X_scaled = pd.concat([X, transformed_df], axis=1)
# Dropping old categorical columns
X_scaled.drop(not_yet_numeric_cols, axis=1, inplace=True)

In [22]:
X_scaled = X_scaled.reset_index(drop=True)
X_scaled.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,onehotencoder__plan_configuration_m,onehotencoder__plan_configuration_n,onehotencoder__plan_configuration_o,onehotencoder__plan_configuration_q,onehotencoder__plan_configuration_s,onehotencoder__plan_configuration_u,onehotencoder__legal_ownership_status_a,onehotencoder__legal_ownership_status_r,onehotencoder__legal_ownership_status_v,onehotencoder__legal_ownership_status_w
0,-0.983414,-0.518705,1.629055,-0.178274,0.047100,-0.459460,-0.226419,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.734459,0.481998,-0.945017,-0.178274,-0.224765,-0.004110,0.816109,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.883744,-0.819158,0.744612,-0.178274,-0.224765,-0.687135,-0.226419,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.008221,-0.685893,1.216589,-0.178274,-0.224765,-0.459460,-0.226419,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.361028,-1.381296,-1.308119,1.195989,0.047100,-0.004110,1.858636,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Full Testing Data

In [23]:
transformed_test = transformer.transform(test_values)
dense_array = transformed_test.toarray()
transformed_df = pd.DataFrame(dense_array, columns = transformer.get_feature_names_out())
transformed_df.index = test_values.index

# Joining tables
X_test_scaled = pd.concat([test_values, transformed_df], axis=1)
# Dropping old categorical columns
X_test_scaled.drop(not_yet_numeric_cols, axis=1, inplace=True)

In [24]:
X_test_scaled = X_test_scaled.reset_index(drop=True)
X_test_scaled.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,onehotencoder__plan_configuration_m,onehotencoder__plan_configuration_n,onehotencoder__plan_configuration_o,onehotencoder__plan_configuration_q,onehotencoder__plan_configuration_s,onehotencoder__plan_configuration_u,onehotencoder__legal_ownership_status_a,onehotencoder__legal_ownership_status_r,onehotencoder__legal_ownership_status_v,onehotencoder__legal_ownership_status_w
0,0.387545,-0.261582,1.382754,1.190666,-0.089223,-0.231598,0.295710,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.982399,-1.362313,1.569122,-0.182880,-0.021116,1.138930,-0.228690,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.010246,-1.657455,1.036602,-0.182880,-0.293547,-0.916862,-0.228690,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.508407,-1.609071,-1.542679,-1.556426,-0.361655,2.509459,-1.277491,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.387545,-1.004273,0.468179,1.190666,-0.157331,-0.003176,0.820111,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Model Building

In [25]:
# Level 0
estimators = []
estimators.append(('AdaBoostClassifier', AdaBoostClassifier(random_state=13)))
estimators.append(('Bagging Classifier', BaggingClassifier(n_jobs=1, random_state=13)))
# estimators.append(('Bernoulli NB', BernoulliNB()))
# estimators.append(('Decision Tree Classifier', DecisionTreeClassifier(random_state=13)))
# estimators.append(('Dummy Classifier', DummyClassifier(random_state=13)))
# estimators.append(('Extra Tree Classifier', ExtraTreeClassifier(random_state=13)))
# estimators.append(('Extra Trees Classifier', ExtraTreesClassifier(n_jobs=1, random_state=13)))
# estimators.append(('Gaussian NB', GaussianNB()))
# estimators.append(('Gaussian Process Classifier', GaussianProcessClassifier(random_state=13)))
estimators.append(('Gradient Boosting Classifier', GradientBoostingClassifier(n_estimators=100, random_state=13)))
estimators.append(('Hist Gradient Boosting Classifier', HistGradientBoostingClassifier(random_state=13)))
estimators.append(('KNN', KNeighborsClassifier(n_jobs=1)))
# estimators.append(('Label Propagation', LabelPropagation()))
# estimators.append(('Label Spreading', LabelSpreading()))
estimators.append(('LogisticRegression', LogisticRegression(max_iter=1000, random_state=13)))
# estimators.append(('Logistic Regression CV', LogisticRegressionCV(max_iter=1000, random_state=13)))
# estimators.append(('MLPClassifier', MLPClassifier(max_iter=1000, random_state=13)))
# estimators.append(('Nearest Centroid', NearestCentroid()))
# estimators.append(('Passive Aggressive Classifier', PassiveAggressiveClassifier(random_state=13)))
# estimators.append(('Perceptron', Perceptron(random_state=13)))
# estimators.append(('RadiusNeighborsClassifier', RadiusNeighborsClassifier(radius=3)))
estimators.append(('RandomForest', RandomForestClassifier(max_depth=10, min_samples_leaf=1, min_samples_split=3, n_estimators=100, random_state=13)))
# estimators.append(('Ridge Classifier', RidgeClassifier(random_state=13)))
# estimators.append(('Ridge Classifier CV', RidgeClassifierCV()))
# estimators.append(('SGDClassifier', SGDClassifier(random_state=13)))
estimators.append(('SVC', SVC(random_state=13)))
# estimators.append(('XGB', XGBClassifier(random_state=13)))
# estimators.append(('CatBoost', CatBoostClassifier(logging_level='Silent', random_state=13)))

In [26]:
print(estimators)

[('AdaBoostClassifier', AdaBoostClassifier(random_state=13)), ('Bagging Classifier', BaggingClassifier(n_jobs=1, random_state=13)), ('Gradient Boosting Classifier', GradientBoostingClassifier(random_state=13)), ('Hist Gradient Boosting Classifier', HistGradientBoostingClassifier(random_state=13)), ('KNN', KNeighborsClassifier(n_jobs=1)), ('LogisticRegression', LogisticRegression(max_iter=1000, random_state=13)), ('RandomForest', RandomForestClassifier(max_depth=10, min_samples_split=3, random_state=13)), ('SVC', SVC(random_state=13))]


In [27]:
# Level 1
HGB = HistGradientBoostingClassifier(random_state=13)

In [28]:
# Stacking Classifier
SC = StackingClassifier(estimators=estimators,final_estimator=HGB,cv=6)
SC.fit(xtrain, ytrain)
y_pred = SC.predict(xtest)

print(f"\nStacking classifier training Accuracy: {SC.score(xtrain, ytrain):0.2f}")
print(f"Stacking classifier test Accuracy: {SC.score(xtest, ytest):0.2f}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_hist_gradient_boosting/gradient_boosting.py:400: FutureWarning: The categorical_features parameter will change to 'from_dtype' in v1.6. The 'from_dtype' option automatically treats categorical dtypes in a DataFrame as categorical features.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packag

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, cross_val_score

SC_Recall = recall_score(ytest, y_pred)
SC_Precision = precision_score(ytest, y_pred)
SC_f1 = f1_score(ytest, y_pred)
SC_accuracy = accuracy_score(ytest, y_pred)
SC_roc_auc = roc_auc_score(ytest, y_pred)

cm = confusion_matrix(ytest, y_pred)
print(cm)

In [ ]:
from statistics import stdev
score = cross_val_score(SC, xtrain, ytrain, cv=5, scoring='recall', error_score="raise")
SC_cv_score = score.mean()
SC_cv_stdev = stdev(score)
print('Cross Validation Recall scores are: {}'.format(score))
print('Average Cross Validation Recall score: ', SC_cv_score)
print('Cross Validation Recall standard deviation: ', SC_cv_stdev)

In [ ]:
y_proba = SC.predict_proba(xtest)

from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
def plot_auc_roc_curve(y_test, y_pred):
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
    roc_display.figure_.set_size_inches(5,5)
    plt.plot([0, 1], [0, 1], color = 'g')
plot_auc_roc_curve(ytest, y_proba[:, 1])

### Final Predictions

In [ ]:
# Stacking Classifier
SC = StackingClassifier(estimators=estimators,final_estimator=HGB,cv=6)
SC.fit(X_scaled, y)
y_pred = SC.predict(X_test_scaled)

In [ ]:
submission = pd.DataFrame({
    'building_id' : test_values['building_id'],
    'damage_grade' : y_pred
})